Applied Data Science Capstone

Segmenting and Clustering Neighborhoods in Toronto

by
Marcos Cifuentes
    
    

INSTALL LIBRARIES beautifulsoup4,lxml

In [15]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [202]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import geocoder

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values


# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors


Solving environment: ...working... done

# All requested packages already installed.



In [167]:
##  requests.get => retriving  information about /List_of_postal_codes_of_Canada
## source takes <!DOCTYPE html> 
## soup SAVE HTML WITH LABELS
## table_contents SAVE CELL BY CELL AFTER ITERATION


1. Download Dataset
From the Wikipage given the List of postal codes of Canada :

Download the HTML file at the given link : https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Register the file locally

Open the file and iterate through HTML elements to extract postal codes using 'BeautifulSoup' library

In [21]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')
table = soup.find_all('table')[0] 
table
table_contents=[]
for row in table.findAll('td'):
    cell = {}
  #  print("===>",''+row.span.text,row.p.text[:3])
  
    if row.span.text=='Not assigned':        
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = 'Not assigned'
        cell['Neighborhood'] = 'Not assigned'
        table_contents.append(cell)
        pass
    else:
        cell_temp=row.span.text,row.p.text[:3]        
       # print("| "+row.span.text,row.p.text[:3]+ " | " + (row.span.text).split('(')[0] + " | " + ((((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' '))+ " | " + row.p.text[:3])

        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        
       # print(cell['PostalCode'])
       # print(cell['Borough'])
        #print(cell['Neighborhood'])
        table_contents.append(cell)

#print(table_contents)
df=pd.DataFrame(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                            'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.shape
df.head()
df[0:10]

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"


In [ ]:
#Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, 
#in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

In [22]:
print('There are {} uniques postal codes.'.format(len(df['PostalCode'].unique())))

There are 180 uniques postal codes.


Here we will use Nominatim already imported in the beginning of the file

I tryied with Geocoder and Nominatim : no results, the program loops without giving more than 1 coordinates. So let's use the csv file.

In [ ]:
#import geocoder
#for index, row in df.iterrows():
#    print(row['PostalCode'], row['Borough'], row['Neighborhood'])
#    address = row['Borough']+ ', '+row['Neighborhood']
#    geolocator = Nominatim(user_agent="explorer")
#    lat_lng_coords = geolocator.geocode(address)
#    print('=>',lat_lng_coords)    
#    # loop until you get the coordinates
#    while(lat_lng_coords != None):
#        g = geocoder.google('{},'.format(row['PostalCode']))
#        lat_lng_coords = g.latlng
#        print('lat_lng_coords=>',lat_lng_coords)    
#    latitude = lat_lng_coords[0]
#    longitude = lat_lng_coords[1]




In [23]:
#!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data   
print('Postal Code Coordinates downloaded!')

df_coordinates = pd.read_csv('Geospatial_Coordinates.csv')


Postal Code Coordinates downloaded!


In [24]:
# Rename the Postal Code column to allow merging
df_coordinates.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
print(df_coordinates[0:3])
# for each postal code get the latitude and longitude values
# Merge the 2 dtaframe on the Postalcode column


# for each postal code get the latitude and longitude values
# Merge the 2 dtaframe on the Postalcode column
df.head(10)
df_latlg = pd.merge(df, df_coordinates, on='PostalCode' )
df_latlg.head(50)


  PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
